In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import graphviz
from sklearn.linear_model import LassoCV

In [3]:
print("Maint hours:\n", pd.read_csv("./data/PdM_maint.csv", parse_dates=["datetime"])["datetime"].dt.hour.value_counts().sort_index())
print("Fail hours:\n",  pd.read_csv("./data/PdM_failures.csv", parse_dates=["datetime"])["datetime"].dt.hour.value_counts().sort_index())
print("Err hours:\n",   pd.read_csv("./data/PdM_errors.csv", parse_dates=["datetime"])["datetime"].dt.hour.value_counts().sort_index())

Maint hours:
 6    3286
Name: datetime, dtype: int64
Fail hours:
 3     18
6    743
Name: datetime, dtype: int64
Err hours:
 0      126
1      132
2      142
3      112
4      127
5      117
6     1122
7      119
8      135
9      113
10     124
11      98
12     120
13     130
14     128
15     129
16     113
17     124
18     127
19     108
20     127
21     108
22     116
23     122
Name: datetime, dtype: int64


In [4]:
def load_data_daily(
    telemetry_path: str,
    errors_path: str,
    failures_path: str,
    maint_path: str,
    machines_path: str,
    machine_id: int = 0,
    scale_sensors: bool = True,
    telemetry_agg: str = "mean",  # could be "mean", "median", etc.
) -> pd.DataFrame:
    
    def add_operational_day(df: pd.DataFrame) -> pd.DataFrame:
        # shift back so 06:00 belongs to the same day as the preceding hours
        shift = pd.Timedelta(hours=6)
        df = df.copy()
        df["day"] = (df["datetime"] - shift).dt.floor("D")
        return df

    def process_events(path: str, col_prefix_map: dict) -> pd.DataFrame:
        df = pd.read_csv(path, parse_dates=["datetime"])
        df = add_operational_day(df)

        df = pd.get_dummies(df, dtype=int)

        df = df.groupby(["machineID", "datetime", "day"], as_index=False).max()

        event_cols = [c for c in df.columns if any(c.startswith(k) for k in col_prefix_map.keys())]
        df = df.groupby(["machineID", "day"], as_index=False)[event_cols].max()

        rename_map = {}
        for prefix, pretty in col_prefix_map.items():
            for c in event_cols:
                if c.startswith(prefix):
                    rename_map[c] = c.replace(prefix, pretty)
        df = df.rename(columns=rename_map)

        for c in df.columns:
            if c not in ["machineID", "day"]:
                df[c] = df[c].astype(int)

        return df

    tele = pd.read_csv(telemetry_path, parse_dates=["datetime"])
    tele = add_operational_day(tele)

    sensor_cols = ["volt", "rotate", "pressure", "vibration"]
    tele_daily = (tele.groupby(["machineID", "day"], as_index=False)[sensor_cols]
                     .agg(telemetry_agg))

    err_daily  = process_events(errors_path,   {"errorID_": "error_"})
    fail_daily = process_events(failures_path, {"failure_": "failure_"})
    maint_daily= process_events(maint_path,    {"comp_": "maint_"})

    df = tele_daily.merge(err_daily,  on=["machineID", "day"], how="left") \
                   .merge(fail_daily, on=["machineID", "day"], how="left") \
                   .merge(maint_daily,on=["machineID", "day"], how="left")

    # fill missing events with 0
    event_cols = [c for c in df.columns if c.startswith(("error_", "failure_", "maint_"))]
    df[event_cols] = df[event_cols].fillna(0).astype(int)

    rename = {
    "error_error1":"error_1","error_error2":"error_2","error_error3":"error_3","error_error4":"error_4","error_error5":"error_5",
    "failure_comp1":"failure_c1","failure_comp2":"failure_c2","failure_comp3":"failure_c3","failure_comp4":"failure_c4",
    "maint_comp1":"maint_c1","maint_comp2":"maint_c2","maint_comp3":"maint_c3","maint_comp4":"maint_c4",
    }
    df = df.rename(columns=rename)


    machines = pd.read_csv(machines_path)
    df = df.merge(machines, on="machineID", how="left")

    if scale_sensors:
        scaler = StandardScaler()
        for mid in df["machineID"].unique():
            m = df["machineID"] == mid
            df.loc[m, sensor_cols] = scaler.fit_transform(df.loc[m, sensor_cols])

    df = df.sort_values(["machineID", "day"]).set_index(["machineID", "day"])

    if machine_id != 0:
        df = df.loc[(machine_id, slice(None)), :]

    return df

In [5]:
data = load_data_daily(
    telemetry_path="./data/PdM_telemetry.csv",
    errors_path="./data/PdM_errors.csv",
    failures_path="./data/PdM_failures.csv",
    maint_path="./data/PdM_maint.csv",
    machines_path="./data/PdM_machines.csv",
    machine_id=0,  # all machines
    scale_sensors=True,
    telemetry_agg="mean",
)
data.head()

volt    rotate  pressure  vibration  error_1  \
machineID day                                                            
1         2015-01-01 -0.257097 -0.056867 -0.898499  -0.080730        0   
          2015-01-02 -0.118429  0.463672  0.027563  -0.876025        0   
          2015-01-03  0.373405  0.271524 -0.099438   4.893198        1   
          2015-01-04  0.098945 -0.279084 -0.599137   4.584068        0   
          2015-01-05  0.112280  1.274486  0.877956  -0.241107        0   

                      error_2  error_3  error_4  error_5  failure_c1  \
machineID day                                                          
1         2015-01-01        0        0        0        0           0   
          2015-01-02        0        0        0        0           0   
          2015-01-03        0        1        0        0           0   
          2015-01-04        0        0        0        1           0   
          2015-01-05        0        0        0        0           0   

                      failure_c2  failure_c3  failure_c4  maint_c1  maint_c2  \
machineID day                                                                  
1         2015-01-01           0           0           0         0         0   
          2015-01-02           0           0           0         0         0   
          2015-01-03           0           0           0         0         0   
          2015-01-04           0           0           0         0         0   
          2015-01-05           0           0           1         1         0   

                      maint_c3  maint_c4   model  age  
machineID day                                          
1         2015-01-01         0         0  model3   18  
          2015-01-02         0         0  model3   18  
          2015-01-03         0         0  model3   18  
          2015-01-04         0         0  model3   18  
          2015-01-05         0         1  model3   18

In [6]:
from tigramite import data_processing as pp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.regressionCI import RegressionCI


def run_pcmciplus(
    df_daily: pd.DataFrame,
    tau_max: int = 7,
    tau_min: int = 0,
    pc_alpha: float = 0.05,
    fdr_method: str = "fdr_bh",
    max_machines: int = 5,
    n_days: int = 100,
    verbosity: int = 1,
    model_filter: str | None = None,
):

    d = df_daily.reset_index().copy()
    d["day"] = pd.to_datetime(d["day"])
    d = d.sort_values(["machineID", "day"])

    if model_filter is not None:
        d = d[d["model"] == model_filter].copy()

    static_cols = {"machineID", "day", "model", "age"}
    var_names = [c for c in d.columns if c not in static_cols]

    continuous = {"volt", "rotate", "pressure", "vibration"}
    var_type = np.array([0 if c in continuous else 1 for c in var_names], dtype=int)

    start_day = d["day"].min()
    days = pd.date_range(start_day, periods=n_days, freq="D")

    data_dict = {}
    data_type_dict = {}

    machine_ids = d["machineID"].drop_duplicates().head(max_machines).tolist()
    kept_machine_ids = []

    for mid in machine_ids:
        dm = (d[d["machineID"] == mid]
              .set_index("day")[var_names]
              .sort_index())

        X = dm.loc[days].to_numpy()
        if np.isnan(X).any():
            continue

        data_dict[mid] = X
        data_type_dict[mid] = np.tile(var_type, (X.shape[0], 1))
        kept_machine_ids.append(mid)


    X_all = np.vstack([data_dict[mid] for mid in kept_machine_ids])
    nonconstant = (X_all.max(axis=0) != X_all.min(axis=0))
    var_names = [v for v, keep in zip(var_names, nonconstant) if keep]
    var_type = var_type[nonconstant]

    for mid in kept_machine_ids:
        data_dict[mid] = data_dict[mid][:, nonconstant]
        data_type_dict[mid] = np.tile(var_type, (data_dict[mid].shape[0], 1))

    dataframe = pp.DataFrame(
        data=data_dict,
        data_type=data_type_dict,
        var_names=var_names,
        analysis_mode="multiple",
    )

    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=RegressionCI(), verbosity=verbosity)

    results = pcmci.run_pcmciplus(
        tau_min=tau_min,
        tau_max=tau_max,
        pc_alpha=pc_alpha,
        conflict_resolution=True,
        fdr_method=fdr_method,
    )

    graph = results["graph"]
    N = len(var_names)

    Gs = nx.DiGraph()
    Gs.add_nodes_from(var_names)

    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            lags = []
            for tau in range(tau_min, tau_max + 1):
                if graph[i, j, tau] == "-->":
                    lags.append(tau)
            if lags:
                Gs.add_edge(var_names[i], var_names[j], lags=sorted(lags), method="PCMCI+")

    return Gs, results, var_names


In [9]:
Gs, results, var_names = run_pcmciplus(
    df_daily=data,
    tau_max=14,
    tau_min=1,
    max_machines=5,
    n_days=365,
    model_filter="model1",
    verbosity=1,
)

print("Nodes:", Gs.number_of_nodes())
print("Edges:", Gs.number_of_edges())
print("First 15 edges:")
for u, v, d in list(Gs.edges(data=True))[:15]:
    print(f"{u} -> {v}  lags={d['lags']}")



##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = regression_ci
tau_min = 1
tau_max = 14
pc_alpha = [0.05]
max_conds_dim = None
max_combinations = 1



## Resulting lagged parent (super)sets:

    Variable volt has 5 link(s):
        (volt -1): max_pval = 0.00000, |min_val| =  218.886
        (maint_c1 -7): max_pval = 0.00048, |min_val| =  15.283
        (error_4 -14): max_pval = 0.02830, |min_val| =  7.129
        (maint_c1 -6): max_pval = 0.04315, |min_val| =  6.286
        (volt -5): max_pval = 0.02829, |min_val| =  4.810

    Variable rotate has 9 link(s):
        (rotate -1): max_pval = 0.00000, |min_val| =  298.223
        (maint_c1 -13): max_pval = 0.00000, |min_val| =  30.704
        (maint_c3 -13): max_pval = 0.00440, |min_val| =  10.853
        (maint_c4 -13): max_pval = 0.00796, |min_val| =  9.668
        (maint_c1 -14): max_pval = 0.02378, |min_val| =  7.478
        (failure_c1 -1): max_pval = 0.02898, |min_val| =  7.082
      

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit


def run_gcmvl(
    df_daily: pd.DataFrame,
    tau_max: int = 7,
    model_filter: str | None = None,
    max_machines: int = 5,
    n_days: int = 120,
    continuous_vars: set[str] = {"volt", "rotate", "pressure", "vibration"},
    coef_threshold: float = 1e-5,
    use_logit_cv: bool = True,
    n_splits: int = 5,
    include_self: bool = True,
    random_state: int = 0,
    verbosity: int = 0,
):
    def _build_lagged_XY(values: np.ndarray, p: int):
        T, N = values.shape
        if T <= p:
            return None, None
        X = np.zeros((T - p, N * p), dtype=float)
        for lag in range(1, p + 1):
            X[:, (lag - 1) * N : lag * N] = values[p - lag : T - lag, :]
        Y = values[p:, :]
        return X, Y

    d = df_daily.reset_index().copy()
    d["day"] = pd.to_datetime(d["day"])
    d = d.sort_values(["machineID", "day"])

    if model_filter is not None:
        d = d[d["model"] == model_filter].copy()

    static_cols = {"machineID", "day", "model", "age"}
    var_names = [c for c in d.columns if c not in static_cols]
    N = len(var_names)

    start_day = d["day"].min()
    days = pd.date_range(start_day, periods=n_days, freq="D")

    X_blocks, Y_blocks = [], []
    block_starts = []
    block_len = None

    kept = []
    for mid in d["machineID"].unique():
        if len(kept) >= max_machines:
            break

        dm = (d[d["machineID"] == mid]
              .set_index("day")[var_names]
              .reindex(days))

        if dm.isna().any().any():
            continue

        values = dm.to_numpy(dtype=float)
        X_m, Y_m = _build_lagged_XY(values, tau_max)
        if X_m is None:
            continue

        if block_len is None:
            block_len = X_m.shape[0]
        else:
            if X_m.shape[0] != block_len:
                continue

        block_starts.append(sum(b.shape[0] for b in X_blocks))
        X_blocks.append(X_m)
        Y_blocks.append(Y_m)
        kept.append(mid)

    if len(kept) == 0:
        raise ValueError("No machines had a complete daily window for the chosen slice.")

    X = np.vstack(X_blocks)   # (M*L, N*p)
    Y = np.vstack(Y_blocks)   # (M*L, N)
    L = block_len

    tscv = TimeSeriesSplit(n_splits=min(n_splits, max(2, L - 1)))

    panel_folds = []
    for train_local, test_local in tscv.split(np.arange(L)):
        train_idx = np.concatenate([start + train_local for start in block_starts])
        test_idx  = np.concatenate([start + test_local  for start in block_starts])
        panel_folds.append((train_idx, test_idx))

    Gs = nx.DiGraph()
    Gs.add_nodes_from(var_names)
    models = {}

    def coef_to_lag_matrix(coef_vec: np.ndarray) -> np.ndarray:
        return coef_vec.reshape(tau_max, N)  # rows: lag 1..p, cols: source vars

    for j, target in enumerate(var_names):
        y = Y[:, j]
        is_cont = target in continuous_vars

        if is_cont:
            est = Pipeline([
                ("scaler", StandardScaler()),
                ("model", LassoCV(cv=panel_folds, alphas=50, max_iter=5000, random_state=random_state))
            ])
        else:
            classes = np.unique(y)
            if classes.size < 2:
                if verbosity:
                    print(f"[skip] {target}: only one class in slice")
                continue

            if use_logit_cv:
                est = Pipeline([
                    ("scaler", StandardScaler()),
                    ("model", LogisticRegressionCV(
                        cv=panel_folds,
                        penalty="l1",
                        solver="saga",
                        scoring="neg_log_loss",
                        max_iter=5000,
                        n_jobs=-1,
                        random_state=random_state
                    ))
                ])
            else:
                est = Pipeline([
                    ("scaler", StandardScaler()),
                    ("model", LogisticRegression(
                        penalty="l1",
                        solver="saga",
                        C=0.01,
                        max_iter=5000,
                        n_jobs=-1,
                        random_state=random_state
                    ))
                ])

        try:
            est.fit(X, y)
        except Exception as e:
            if not is_cont and use_logit_cv:
                if verbosity:
                    print(f"[warn] {target}: LogisticRegressionCV failed ({e}); falling back to fixed-C.")
                est = Pipeline([
                    ("scaler", StandardScaler()),
                    ("model", LogisticRegression(
                        penalty="l1", solver="saga", C=0.1,
                        max_iter=5000, n_jobs=-1, random_state=random_state
                    ))
                ])
                est.fit(X, y)
            else:
                raise

        models[target] = est

        model = est.named_steps["model"]
        coef = model.coef_.ravel() if hasattr(model, "coef_") else model.coef_.ravel()
        lag_mat = coef_to_lag_matrix(coef)

        for i, source in enumerate(var_names):
            if (not include_self) and (i == j):
                continue

            nz_lags = [lag for lag in range(1, tau_max + 1)
                       if abs(lag_mat[lag - 1, i]) > coef_threshold]

            if nz_lags:
                if Gs.has_edge(source, target):
                    Gs[source][target]["lags"] = sorted(set(Gs[source][target]["lags"]).union(nz_lags))
                else:
                    Gs.add_edge(source, target, lags=sorted(nz_lags), method="GCMVL")

    return Gs, models, var_names


In [14]:
Gs_gcmvl, gcmvl_models, gcmvl_vars = run_gcmvl(
    df_daily=data,
    tau_max=7,
    model_filter="model1",
    max_machines=3,
    n_days=365,
    use_logit_cv=True,
    verbosity=1,
    coef_threshold=0.1
)

print("GCMVL nodes:", Gs_gcmvl.number_of_nodes())
print("GCMVL edges:", Gs_gcmvl.number_of_edges())
print("First 20 edges (source -> target, lags):")
for u, v, d in list(Gs_gcmvl.edges(data=True))[:20]:
    print(f"  {u} -> {v}  lags={d['lags']}")


[warn] error_4: LogisticRegressionCV failed (y_true contains only one label (-1.0). Please provide the list of all expected class labels explicitly through the labels argument.); falling back to fixed-C.
[warn] failure_c2: LogisticRegressionCV failed (y_true contains only one label (-1.0). Please provide the list of all expected class labels explicitly through the labels argument.); falling back to fixed-C.
[warn] failure_c3: LogisticRegressionCV failed (y_true contains only one label (-1.0). Please provide the list of all expected class labels explicitly through the labels argument.); falling back to fixed-C.
GCMVL nodes: 17
GCMVL edges: 102
First 20 edges (source -> target, lags):
  volt -> volt  lags=[1]
  volt -> error_1  lags=[1, 2, 7]
  volt -> error_2  lags=[6]
  volt -> error_3  lags=[1]
  volt -> error_5  lags=[1, 3]
  volt -> failure_c1  lags=[1, 2, 3]
  volt -> maint_c1  lags=[1, 2, 4]
  volt -> maint_c2  lags=[1, 6]
  volt -> maint_c3  lags=[3, 6]
  volt -> maint_c4  lags=[